In [5]:
"""Julia set generator without optional PIL-based image drawing"""
import time
from PIL import Image
import array

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193


def show_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert our output to PIL-compatible input
    # scale to [0...255]
    max_iterations = float(max(output_raw))
    print(max_iterations)
    scale_factor = float(max_iterations)
    scaled = [int(o / scale_factor * 255) for o in output_raw]
    output = array.array('B', scaled)  # array of unsigned ints
    # display with PIL
    im = Image.new("L", (width, width))
    # EXPLAIN RAW L 0 -1
    im.frombytes(output.tobytes(), "raw", "L", 0, -1)
    im.show()


def show_false_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert our output to PIL-compatible input
    assert width * height == len(output_raw)  # sanity check our 1D array and desired 2D form
    # rescale output_raw to be in the inclusive range [0..255]
    max_value = float(max(output_raw))
    output_raw_limited = [int(float(o) / max_value * 255) for o in output_raw]
    # create a slightly fancy colour map that shows colour changes with
    # increased contrast (thanks to John Montgomery)
    output_rgb = ((o + (256 * o) + (256 ** 2) * o) * 16 for o in output_raw_limited)  # fancier
    output_rgb = array.array('I', output_rgb)  # array of unsigned ints (size is platform specific)
    # display with PIL/pillow
    im = Image.new("RGB", (width, height))
    # EXPLAIN RGBX L 0 -1
    im.frombytes(output_rgb.tobytes(), "raw", "RGBX", 0, -1)
    im.show()


def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output


def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # set width and height to the generated pixel counts, rather than the
    # pre-rounding desired width and height
    width = len(x)
    height = len(y)
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    assert sum(output) == 33219980  # this sum is expected for 1000^2 grid with 300 iterations

    if draw_output:
        show_false_greyscale(output, width, height, max_iterations)
        show_greyscale(output, width, height, max_iterations)


if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    # set draw_output to True to use PIL to draw an image
    calc_pure_python(draw_output=True, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 12.081302642822266 seconds
300.0


In [3]:
%timeit -r 5 -n 10 calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)


Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 19.2152099609375 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 16.54414963722229 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 22.47481417655945 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 31.82246732711792 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 26.29069757461548 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 27.48933506011963 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 29.879351377487183 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 19.673796892166138 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 24.811880350112915 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython

KeyboardInterrupt: 

In [4]:
import cProfile
import pstats

# Create a Profile object
profiler = cProfile.Profile()

# Start profiling
profiler.enable()

# Run your function
calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

# Stop profiling
profiler.disable()

# Create a Stats object
stats = pstats.Stats(profiler)

# Clean up filenames for the report
stats.strip_dirs()

# Sort the statistics by the cumulative time spent
stats.sort_stats('time')

# Print out all the statistics
stats.print_stats()

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 55.93590974807739 seconds
         36223471 function calls (36223383 primitive calls) in 59.742 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     15/0   27.851    1.857    0.000          {built-in method select.select}
 34219980   22.294    0.000   22.294    0.000 {built-in method builtins.abs}
        1    5.787    5.787    9.559    9.559 890625955.py:45(calculate_z_serial_purepython)
     15/0    2.527    0.168    0.000          selectors.py:313(_select)
  2002006    1.096    0.000    1.096    0.000 {method 'append' of 'list' objects}
       22    0.096    0.004    0.135    0.006 socket.py:621(send)
        1    0.065    0.065    0.065    0.065 4057984789.py:1(<module>)
        1    0.010    0.010    0.010    0.010 {built-in method builtins.sum}
       18    0.001    0.000    0.001    0.000 threading.py:1220(is_alive)
     15/0    0.001    0.0